<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>

# Laboratorio Agrupamiento de Datos

En este laboratorio usaremos el DataFrame creado en la lección anterior resultado de la unión de los conjuntos de datos compartidos contigo. 

**Instrucciones:**

1. Lee cuidadosamente el enunciado de cada ejercicio.

2. Implementa la solución en la celda de código proporcionada.

3. Devuelve los resultados siempre en formato de DataFrame

In [10]:
import pandas as pd
netflix = pd.read_csv("../Datos/netflix_merged.csv",index_col=0)

In [11]:
netflix.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description',
       'Premiere', 'IMDB Score', 'Language'],
      dtype='object')

#### Ejercicio 1: ¿Cuántos títulos hay por cada tipo de contenido ('Movie' o 'TV Show')?

In [12]:
netflix.groupby("type")["title"].count()

type
Movie      6131
TV Show    2676
Name: title, dtype: int64

#### Ejercicio 2: ¿Cuál es el promedio de puntuación de las películas agrupadas por clasificación de audiencia ('rating')? Interpreta los resultados, ¿hay algo que te llame la atención? Explica por qué. 

In [20]:
netflix.groupby("rating")["IMDB Score"].agg(["mean","count"])

,mean,count
rating,,
G,6.500000,1
NC-17,NaN,0
NR,NaN,0
PG,6.800000,11
PG-13,6.386957,23
R,6.495745,47
TV-14,6.177660,94
TV-G,6.156250,16
TV-MA,6.097638,254


Algunos rating son NaN porque solo tienen rating las peliculas de Netflix Originals, entonces habrá categorías de rating que no tengan valores con los que promediar.

#### Ejercicio 3: ¿Cuántos títulos hay por cada país? Ordena los resultados de mayor a menor

In [19]:
netflix.groupby("country")[["title"]].count().sort_values(by="title",ascending=False)

,title
country,
United States,2818
India,972
United Kingdom,419
Japan,245
South Korea,199
...,...
"Ireland, Canada, Luxembourg, United States, United Kingdom, Philippines, India",1
"Ireland, Canada, United Kingdom, United States",1
"Ireland, Canada, United States, United Kingdom",1


Para contar esto correctamente, habría que explotar los datos. La interpretación sería: ¿Cuántas películas hay asociadas a cada país?

In [25]:
netflix_exploded = netflix.copy()
netflix_exploded["country"] =netflix_exploded["country"].str.split(", ")
netflix_exploded = netflix_exploded.explode("country")
netflix_exploded.groupby("country")[["title"]].count().sort_values(by="title",ascending=False)

,title
country,
United States,3689
India,1046
United Kingdom,804
Canada,445
France,393
...,...
Kazakhstan,1
Jamaica,1
Slovakia,1


#### Ejercicio 4: ¿Cuál es la calificación promedio en IMDb de los títulos agrupados por tipo ('Movie' o 'TV Show')?

In [28]:
netflix.groupby("type")[["IMDB Score"]].agg(["mean","count"])

IMDB Score      
              mean count
type                    
Movie     6.210916   513
TV Show        NaN     0

Los TV Show no tienen puntuación de IMDb porque esas puntuaciones las tienen exclusivamente las peliculas de Netflix Originals.

#### Ejercicio 5: ¿Cuántos títulos se añadieron a Netflix cada año? Para resolver este ejercicio tendrás uqe convertir la columna `date_added` a formato fecha usando el método `pd.to_datetime`. [Aquí](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) tienes la documentación, leeala con detenimiento antes de usar el método. 

In [29]:
netflix[["date_added"]]

,date_added
0,2021-09-25
1,2021-09-24
2,2021-09-24
3,2021-09-24
4,2021-09-24
...,...
8802,2019-11-20
8803,2019-07-01
8804,2019-11-01
8805,2020-01-11


Este procesado se realizó en la práctica anterior, pero el formato hubiera sido:

In [ ]:
netflix["date_added"] = pd.to_datetime(netflix["date_added"], format='%B %d, %Y')

#### Ejercicio 6: ¿Cuál es el país con más títulos lanzados en el año 2020? Para resolver este ejercicio después del gropuby os puede resultar interesante el método [`idxmax`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.idxmax.html)

In [40]:
netflix.query("release_year == 2020").groupby("country")[["title"]].count().idxmax()

title    United States
dtype: object

#### Ejercicio 7:¿Cuál es el promedio de puntuación en IMDb de los títulos agrupados por país?

In [41]:
netflix.groupby("country")[["IMDB Score"]].mean()

,IMDB Score
country,
", France, Algeria",NaN
", South Korea",NaN
Argentina,6.42
"Argentina, Brazil, France, Poland, Germany, Denmark",NaN
"Argentina, Chile",NaN
...,...
Venezuela,NaN
"Venezuela, Colombia",NaN
Vietnam,NaN


In [50]:
netflix_exploded = netflix.copy()
netflix_exploded["country"] =netflix_exploded["country"].str.split(", ")
netflix_exploded = netflix_exploded.explode("country")
netflix_exploded.groupby("country")[["IMDB Score"]].mean().sort_values(by="IMDB Score",ascending=False)[:15]

,IMDB Score
country,
Ukraine,8.400000
"United States,",7.500000
Pakistan,7.300000
Greece,7.300000
Cambodia,7.200000
Ireland,7.150000
Australia,6.966667
Austria,6.850000
United Kingdom,6.828205


#### Ejercicio 8: ¿Cuántos títulos fueron lanzados en cada año agrupados por tipo ('Movie' o 'TV Show')?

In [55]:
netflix_exploded.groupby(["type","release_year"])[["title"]].count().sort_index(level=1,ascending=False)[:20]

,,title
type,release_year,
TV Show,2021,345
Movie,2021,313
TV Show,2020,490
Movie,2020,564
TV Show,2019,445
Movie,2019,763
TV Show,2018,427
Movie,2018,963
TV Show,2017,302


#### Ejercicio 9: ¿Cuál es el país con la mayor cantidad de títulos de clasificación 'TV-MA'?

In [59]:
netflix.query("rating == 'TV-MA'").groupby("country")[["title"]].count().idxmax()

title    United States
dtype: object

#### Ejercicio 10. ¿Cuál es el promedio de duración de las películas agrupadas por país?

In [65]:
netflix_exploded_movies = netflix_exploded.query("type == 'Movie'")
netflix_exploded_movies["duration"] = netflix_exploded_movies["duration"].str.replace(" min","").astype(float)
netflix_exploded_movies.groupby("country")[["duration"]].mean().sort_values(by="duration",ascending=False)[:20]

C:\Users\Miguel\AppData\Local\Temp\ipykernel_25192\2686371739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_exploded_movies["duration"] = netflix_exploded_movies["duration"].str.replace(" min","").astype(float)


,duration
country,
Liechtenstein,200.000000
Slovenia,157.000000
Montenegro,157.000000
Soviet Union,156.666667
Morocco,147.400000
Bahamas,145.000000
Cameroon,143.000000
Kuwait,140.500000
Cambodia,137.000000


#### Ejercicio 11: ¿Cuál es el número total de títulos por país y clasificación de audiencia ('rating')?

In [68]:
netflix.groupby(["country","rating"])[["title"]].count().sort_values(by="title",ascending=False)

title
country                                rating       
United States                          TV-MA     928
India                                  TV-14     550
United States                          R         440
                                       TV-14     406
                                       PG-13     289
...                                              ...
Ireland, United States                 R           1
Ireland, United States, France         TV-14       1
Ireland, United States, United Kingdom TV-Y        1
Israel, Germany                        TV-14       1
Zimbabwe                               TV-G        1

[1172 rows x 1 columns]

In [67]:
netflix_exploded.groupby(["country","rating"])[["title"]].count().sort_values(by="title",ascending=False)

title
country       rating       
United States TV-MA    1100
              R         660
India         TV-14     572
United States TV-14     497
              PG-13     433
...                     ...
Malawi        TV-PG       1
Malaysia      TV-Y        1
Malta         PG-13       1
              R           1
Zimbabwe      TV-MA       1

[516 rows x 1 columns]

#### Ejercicio 12: ¿Cuántos títulos fueron lanzados antes del año 2000 agrupados por país?

In [70]:
netflix.query("release_year < 2000").groupby("country")[["title"]].count().sort_values(by="title",ascending=False)

,title
country,
United States,245
India,81
Hong Kong,29
United Kingdom,18
"United Kingdom, United States",12
...,...
"Argentina, Spain",1
Lebanon,1
"New Zealand, Australia, France, United States",1


#### Ejercicio 13: ¿Cuál es la puntuación promedio en IMDb de los títulos lanzados en 2021 agrupados por país?

In [72]:
netflix.query("release_year == 2021").groupby("country")[["IMDB Score"]].mean().sort_values(by="IMDB Score",ascending=False)

,IMDB Score
country,
"Belgium, Netherlands",7.1
United Kingdom,7.1
"India, United States",7.1
"France, United States",6.5
Spain,6.4
...,...
"United States, Mexico",NaN
"United States, Singapore",NaN
"United States, United Kingdom",NaN


In [73]:
netflix_exploded.query("release_year == 2021").groupby("country")[["IMDB Score"]].mean().sort_values(by="IMDB Score",ascending=False)

,IMDB Score
country,
Belgium,7.100000
United Kingdom,6.700000
Spain,6.400000
Turkey,6.400000
United States,6.285714
India,6.142857
Italy,5.950000
Brazil,5.900000
South Korea,5.866667


#### Ejercicio 14: ¿Cuántos títulos fueron dirigidos por cada director? Ordena los resultados de mayor a menor

Si se refiere a en 2021:

In [74]:
netflix.query("release_year == 2021").groupby("director")[["title"]].count().sort_values(by="title",ascending=False)

,title
director,
Rathindran R Prasad,4
Leigh Janiak,3
Bruno Garotti,2
Keishi Otomo,2
"Chapman Way, Maclain Way",2
...,...
Inma Torrente,1
Ivan Ayr,1
J Blakeson,1


En todos los años:

In [75]:
netflix.groupby("director")[["title"]].count().sort_values(by="title",ascending=False)

,title
director,
Rajiv Chilaka,19
"Raúl Campos, Jan Suter",18
Suhas Kadav,16
Marcus Raboy,16
Jay Karas,14
...,...
Jos Humphrey,1
Jose Gomez,1
Jose Javier Reyes,1


#### Ejercicio 15: ¿Cuál es el lenguaje más frecuente en los títulos lanzados en los últimos 5 años agrupado por país?

In [84]:
import datetime
five_years_prior = datetime.datetime.now().year - 5

In [86]:
netflix.query("release_year >= @five_years_prior").groupby("Language")[["title"]].count().sort_values(by="title",ascending=False)

,title
Language,
English,192
Hindi,25
Spanish,25
French,16
Italian,11
Indonesian,8
Portuguese,7
English/Spanish,5
Korean,5
